In [694]:
# Import postgresql libraries
import psycopg2
from psycopg2 import Error
from psycopg2.extensions import AsIs

# imports
from time import sleep
import math
import requests
import json
import panel as pn
import plotly.express as px
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
from pathlib import Path
from dotenv import load_dotenv
    
import warnings
warnings.filterwarnings('ignore')

In [695]:
# Set up Panel Plotly extension
pn.extension('plotly')

In [696]:
# Import hvplot.pandas after pn.extension
# This avoids plotly initialization failure
import hvplot.pandas

In [272]:
# Define path to datasets
dow_jones_list_path = Path('../data/rawdata/dow_jones_list.json')
sp_500_list_path = Path('../data/rawdata/sp_500_list.json')
nasdaq_100_list_path = Path('../data/rawdata/nasdaq_100_list.json')

In [514]:
# Base URL for fmp
fmp_base_url = 'https://financialmodelingprep.com/api/v3/'

In [515]:
# Read in API keys
load_dotenv('./billy/api_keys.env')
map_box_api = os.getenv("MAPBOX_TOKEN")
fmp_api = os.getenv("FMP_API_KEY")
postgres_user = os.getenv("POSTGRES_USER")
postgres_pwd = os.getenv("POSTGRES_PWD")

px.set_mapbox_access_token(map_box_api)

In [275]:
# Read in datasets
dow_jones_list = pd.read_json (dow_jones_list_path)
sp_500_list = pd.read_json (sp_500_list_path)
nasdaq_100_list = pd.read_json (nasdaq_100_list_path)

# Add MarketCap Column
dow_jones_list['marketcap'] = 0
sp_500_list['marketcap'] = 0
nasdaq_100_list['marketcap'] = 0

In [932]:
# Porfolio holdings
profile_column_names = ['symbol','price','beta','volAvg','mktCap','lastDiv','range','changes','companyName','currency','cik','isin','cusip',
                'exchange','exchangeShortName','industry','website','description','ceo','sector','country','fullTimeEmployees',
                'phone','address','city','state','zip','dcfDiff','dcf','image','ipoDate','defaultImage','isEtf','isActivelyTrading']

quote_column_names = ['symbol','name','price','changesPercentage','change','dayLow','dayHigh','yearHigh','yearLow','marketCap','priceAvg50',
                      'priceAvg200','volume','avgVolume','exchange','open','previousClose','eps','pe','earningsAnnouncement',
                      'sharesOutstanding','timestamp']

column_names = ['symbol','price','beta','volAvg','mktCap','lastDiv','range','changes','companyName','currency','cik','isin','cusip',
                'exchange','exchangeShortName','industry','website','description','ceo','sector','country','fullTimeEmployees',
                'phone','address','city','state','zip','dcfDiff','dcf','image','ipoDate','defaultImage','isEtf','isActivelyTrading',
                'price','changesPercentage','change','dayLow','dayHigh','yearHigh','yearLow','marketCap','priceAvg50','priceAvg200',
                'volume','avgVolume','exchange','open','previousClose','eps','pe','earningsAnnouncement','sharesOutstanding','timestamp',
                'held','totalheld']


# Portfolio Holdings
jorge_holdings = {'AMZN':1,'BA':14,'BABA':10,'DAC':20,'FLGT':141,'PYPL':16}

william_holdings = {'T': 70,'LI': 20,'LNC': 10,'ONTO': 10,'UWMC': 100,'ABBV': 10,'ALL': 10,'GOLD': 100,'BEAM': 10,'BNGO': 70,'CCIV': 145,
                    'CTXS': 10,'GNOG': 220,'NIO': 40,'NVS': 11,'PCAR': 10,'PLTR': 36,'PYPL': 4,'PENN': 8,'PWOD': 45,'RPTX': 10,'RKT': 25,
                    'RBLX': 10,'SGFY': 50,'TER': 8,'VZ': 40}

juancarlos_holdings = {'ZM':150,'MSFT':150,'SQ':150,'BABA':150,'AAPL':150,'TSLA':150,'COST':150,'T':150,'FB':150,'DIS':150,'UBER':150,'JPM':150,'TSN':150,'QCOM':150,'ALLY':150}

In [863]:
# Name to convert big numbers to human readable
millnames = ['',' Thousand',' Million',' Billion',' Trillion']

## Database Functions

In [518]:
def connect(params_dic):
    """ Connect to the PostgreSQL database server """
    conn = None
    try:
        # connect to the PostgreSQL server
        print('Connecting to the PostgreSQL database...')
        conn = psycopg2.connect(**params_dic)
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
        sys.exit(1) 
    print("Connection successful")
    return conn
 
def postgresql_to_dataframe(conn, select_query, columns):
    """
    Tranform a SELECT query into a pandas dataframe
    """
    cursor = conn.cursor()
    try:
        cursor.execute(select_query)
    except (Exception, psycopg2.DatabaseError) as error:
        print("Error: %s" % error)
        cursor.close()
        return 1
    
    # Naturally we get a list of tupples
    tupples = cursor.fetchall()
    cursor.close()
    
    # We just need to turn it into a pandas dataframe
    df = pd.DataFrame(tupples, columns=column_names)
    
    return df

In [519]:
def millify(n):
    n = float(n)
    millidx = max(0,min(len(millnames)-1,
                        int(math.floor(0 if n == 0 else math.log10(abs(n))/3))))

    return '{:.0f}{}'.format(n / 10**(3 * millidx), millnames[millidx])

In [520]:
def append_market_cap(dataset):
    df_marketcap = pd.DataFrame()
    count = 0
    for symbol in dataset['symbol']:
        # Query company info from api
        fmp_profile = (f'market-capitalization/{symbol}?apikey={fmp_api}')
        fmp_profile_response = requests.get(fmp_base_url + fmp_profile)
        data = fmp_profile_response.json()
        marketcap = data[0]['marketCap']
        dataset['marketcap'][count] = marketcap
        count += 1
        sleep(0.05)
        
    return dataset


## Visualization Functions

In [933]:
def sector_distribution_plot(dataset, title):
    pie_plot = px.pie(
        dataset,
        values='marketcap',
        names='sector',
        title=title,
        width=500,
        hover_data={
        #"marketcap": millify(dataset['marketcap']),
        }
    )
    return pie_plot

def porfolio_alloc_plot(dataset, title):
        pie_plot = px.pie(
            data_set,
            values='totalheld',
            names='sector',
            title=title,
            width=500,
            height=500
        )

def portfolio_gather_data(dataset):
    final_df = pd.DataFrame(columns = column_names)
    to_append = []

    # Iterate through positions
    for ticker in list(dataset.keys()): 
        # Query FMP API
        fmp_profile = (f'profile/{ticker}?apikey={fmp_api}')
        fmp_quote = (f'quote/{ticker}?apikey={fmp_api}')
        fmp_profile_response = requests.get(fmp_base_url + fmp_profile)
        fmp_quote_response = requests.get(fmp_base_url + fmp_quote)
        profile_data = fmp_profile_response.json()[0]
        quote_data = fmp_quote_response.json()[0]
        profile_columns = profile_data.keys()
        to_append_profile = [profile_data[column] for column in profile_columns]
        quote_columns = quote_data.keys()
        to_append_quote = [quote_data[column] for column in quote_columns]
        to_append_quote.pop(0)
        to_append_quote.pop(0)
        to_append_calculations = [dataset.get(ticker), dataset.get(ticker)*to_append_quote[0]]
        to_append = to_append_profile + to_append_quote + to_append_calculations 
        df_length = len(final_df)
        final_df.loc[df_length] = to_append          
        
    return final_df

## Other

In [937]:
df_jorge_holdings = portfolio_gather_data(jorge_holdings)
df_william_holdings = portfolio_gather_data(william_holdings)
df_juancarlos_holdings = portfolio_gather_data(juancarlos_holdings)

In [917]:
#dow_jones_list = append_market_cap(dow_jones_list)
#dow_jones_list.to_csv('../data/cleandata/dow_jones_list.csv', index=False)

In [918]:
#nasdaq_100_list = append_market_cap(nasdaq_100_list)
#nasdaq_100_list.to_csv('../data/cleandata/nasdaq_100_list.csv', index=False)

In [919]:
#sp_500_list = append_market_cap(sp_500_list)
#sp_500_list.to_csv('../data/cleandata/sp_500_list.csv', index=False)

In [920]:
# Connection parameters, yours will be different
param_dic = {
    "host"      : "localhost",
    "database"  : "project1_db",
    "user"      : postgres_user,
    "password"  : postgres_pwd
}

In [921]:
# create a connection to the database
conn = connect(param_dic)

Connecting to the PostgreSQL database...
Connection successful


## Dashboard

In [941]:
dow_jones_marketcap = millify(dow_jones_list['marketcap'].sum())
nasdaq_100_marketcap = millify(nasdaq_100_list['marketcap'].sum())
sp_500_marketcap = millify(sp_500_list['marketcap'].sum())
jorge_total = millify(df_jorge_holdings['totalheld'].sum())
william_total = millify(df_william_holdings['totalheld'].sum())
juancarlos_total = millify(df_juancarlos_holdings['totalheld'].sum())

In [944]:
welcome_title = "### This dashboard represents a visual analysis of historical prices of house units, sale price per square foot" \
                " and gross rent in San Francisco, California from 2010 to 2016.  You can navigate through the tabs above to explore" \
                " more details about the evolution of the real estate market in The Golden City across these years."
index_distribution = pn.Column(pn.Row(sector_distribution_plot(dow_jones_list,f'Dow Jones ({dow_jones_list.count()[0]} stocks / {dow_jones_marketcap})'),
                            sector_distribution_plot(nasdaq_100_list,f'Nasdaq ({nasdaq_100_list.count()[0]} stocks / {nasdaq_100_marketcap})'),
                            sector_distribution_plot(sp_500_list,f'S&P 500 ({sp_500_list.count()[0]} stocks / {sp_500_marketcap})')),
                               pn.Row(portfolio_alloc_plot(jorge_holdings, f"Jorge's Portfolio ({len(jorge_holdings)} stocks / {jorge_total})"),
                                      portfolio_alloc_plot(william_holdings, f"William's Portfolio ({len(william_holdings)} stocks / {william_total})"),
                                      portfolio_alloc_plot(juancarlos_holdings, f"Juan's Portfolio ({len(juancarlos_holdings)} stocks / {juancarlos_total})")))


In [945]:
# Create a Title for the Dashboard
dashboard_title = '# Index Analysis'

# Create a tab layout for the dashboard
index_analysis_dashboard = pn.Column(pn.pane.Markdown(dashboard_title, width=700),pn.Tabs(('Allocation by Sector', index_distribution)))                                    
                                       
# Create the dashboard
index_analysis_dashboard.servable()

Column
    [0] Markdown(str, width=700)
    [1] Tabs
        [0] Column
            [0] Row
                [0] Plotly(Figure)
                [1] Plotly(Figure)
                [2] Plotly(Figure)
            [1] Row
                [0] Plotly(Figure)
                [1] Plotly(Figure)
                [2] Plotly(Figure)